In [257]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 1000)

In [8]:
train = pd.read_csv('./data/train/train.csv')
print(train.shape)
train.head(1)

(52560, 9)


,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0


#### SunnyTime 변수 추가 (해가 떠있는 시간)

In [71]:
def time_minute(data):
    hour = data['Hour']
    if data['Minute'] == 0:
        minute = 0
    else:
        minute = 0.5
    
    return hour + minute


ghi = train.iloc[np.where(train['DHI'] + train['DNI'] >0)[0]].reset_index(drop=True)
ghi['time_float'] = [time_minute(ghi.iloc[i]) for i in range(len(ghi.index))]

In [72]:
ghi

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,time_float
0,0,8,0,29,494,1.8,65.45,-9,7.039287,8.0
1,0,8,30,61,7,1.9,55.90,-7,5.912871,8.5
2,0,9,0,58,743,2.1,57.39,-6,22.337268,9.0
3,0,9,30,67,811,1.9,53.15,-4,29.469529,9.5
4,0,10,0,138,368,1.8,55.99,-3,25.339762,10.0
...,...,...,...,...,...,...,...,...,...,...
25895,1094,14,30,93,538,2.9,43.02,0,24.869688,14.5
25896,1094,15,0,73,516,2.8,46.51,0,19.426511,15.0
25897,1094,15,30,37,756,2.4,50.11,-2,17.080689,15.5
25898,1094,16,0,28,607,2.0,59.31,-4,9.197496,16.0


In [73]:
train_by_day = pd.DataFrame(ghi.groupby(['Day'])['time_float'].min()).reset_index()
train_by_day.rename(columns = {'time_float': 'sunrise'}, inplace=True)
train_by_day['sunset'] = pd.DataFrame(ghi.groupby(['Day'])['time_float'].max())['time_float']
train_by_day['sunnyTime'] = train_by_day['sunset'] - train_by_day['sunrise']

In [74]:
print(train_by_day.shape)
train_by_day.head(1)

(1095, 4)


,Day,sunrise,sunset,sunnyTime
0,0,8.0,16.5,8.5


In [75]:
train_by_day[train_by_day['sunnyTime'] == 8] # 마지막 동지를 2018년 동지, 12월22일로 지정 (1월1일부터 데이터 맞음)

,Day,sunrise,sunset,sunnyTime
356,356,8.0,16.0,8.0
721,721,8.0,16.0,8.0
1086,1086,8.0,16.0,8.0


In [76]:
train_by_day['date'] = train_by_day['Day']%365

In [139]:
def day_to_month(date):
    
    if date < 31 :
        return 0
    elif date < 59 :
        return 1
    elif date < 90 :
        return 2
    elif date < 120 :
        return 3
    elif date < 151 :
        return 4
    elif date < 181 :
        return 5
    elif date < 212 :
        return 6
    elif date < 243 :
        return 7
    elif date < 273 :
        return 8
    elif date < 304 :
        return 9
    elif date < 334 :
        return 10
    else :
        return 11

In [140]:
train_by_day['month'] = [day_to_month(i) for i in train_by_day['date']]

In [141]:
train_by_day

,Day,sunrise,sunset,sunnyTime,date,month,T_mean,DHI_mean,DNI_mean,WS_mean,RH_mean,TARGET_mean
0,0,8.0,16.5,8.5,0,0,-7.979167,44.937500,78.708333,1.929167,70.329375,6.520751
1,1,8.0,16.5,8.5,1,0,-6.312500,18.604167,295.187500,1.718750,74.231250,11.025184
2,2,8.0,16.5,8.5,2,0,-6.479167,28.937500,24.645833,2.470833,76.275000,3.165478
3,3,8.0,16.5,8.5,3,0,-5.687500,39.312500,66.979167,2.279167,65.695208,5.810807
4,4,8.0,16.5,8.5,4,0,0.854167,44.125000,22.500000,3.995833,73.361042,4.817273
...,...,...,...,...,...,...,...,...,...,...,...,...
1090,1090,8.0,16.5,8.5,360,11,-0.895833,18.729167,306.437500,2.810417,57.857500,11.261230
1091,1091,8.0,16.5,8.5,361,11,0.854167,29.833333,206.583333,1.970833,55.269792,9.349126
1092,1092,8.0,16.5,8.5,362,11,2.187500,34.562500,171.291667,3.368750,49.227292,8.875729
1093,1093,8.0,16.5,8.5,363,11,1.687500,16.708333,309.812500,3.252083,49.218333,11.114171


In [142]:
train_by_day['T_mean'] = pd.DataFrame(train.groupby(['Day'])['T'].mean())['T']
train_by_day['DHI_mean'] = pd.DataFrame(train.groupby(['Day'])['DHI'].mean())['DHI']
train_by_day['DNI_mean'] = pd.DataFrame(train.groupby(['Day'])['DNI'].mean())['DNI']
train_by_day['WS_mean'] = pd.DataFrame(train.groupby(['Day'])['WS'].mean())['WS']
train_by_day['RH_mean'] = pd.DataFrame(train.groupby(['Day'])['RH'].mean())['RH']
train_by_day['TARGET_mean'] = pd.DataFrame(train.groupby(['Day'])['TARGET'].mean())['TARGET']

In [143]:
train_by_day

,Day,sunrise,sunset,sunnyTime,date,month,T_mean,DHI_mean,DNI_mean,WS_mean,RH_mean,TARGET_mean
0,0,8.0,16.5,8.5,0,0,-7.979167,44.937500,78.708333,1.929167,70.329375,6.520751
1,1,8.0,16.5,8.5,1,0,-6.312500,18.604167,295.187500,1.718750,74.231250,11.025184
2,2,8.0,16.5,8.5,2,0,-6.479167,28.937500,24.645833,2.470833,76.275000,3.165478
3,3,8.0,16.5,8.5,3,0,-5.687500,39.312500,66.979167,2.279167,65.695208,5.810807
4,4,8.0,16.5,8.5,4,0,0.854167,44.125000,22.500000,3.995833,73.361042,4.817273
...,...,...,...,...,...,...,...,...,...,...,...,...
1090,1090,8.0,16.5,8.5,360,11,-0.895833,18.729167,306.437500,2.810417,57.857500,11.261230
1091,1091,8.0,16.5,8.5,361,11,0.854167,29.833333,206.583333,1.970833,55.269792,9.349126
1092,1092,8.0,16.5,8.5,362,11,2.187500,34.562500,171.291667,3.368750,49.227292,8.875729
1093,1093,8.0,16.5,8.5,363,11,1.687500,16.708333,309.812500,3.252083,49.218333,11.114171


### train_by_day 데이터를 통해 month를 예측하는 모델 수립

In [307]:
import xgboost as xgb
import lightgbm as lgbm

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [308]:
X = train_by_day.drop(['Day', 'date', 'month'], axis=1)
y = train_by_day['month']

In [309]:
SEED = 777
TEST_RATIO = 0.2

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_RATIO, random_state=SEED)

In [310]:
len(X_train), len(X_val)

(876, 219)

In [330]:
model = xgb.XGBClassifier(use_label_encoder=False, objective='multi:softmax', eval_metric='mlogloss', n_estimators=1000, random_state=SEED)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=200)

[0]	validation_0-mlogloss:1.61345
[1]	validation_0-mlogloss:1.29883
[2]	validation_0-mlogloss:1.09609
[3]	validation_0-mlogloss:0.95269
[4]	validation_0-mlogloss:0.85027
[5]	validation_0-mlogloss:0.76416
[6]	validation_0-mlogloss:0.70184
[7]	validation_0-mlogloss:0.65565
[8]	validation_0-mlogloss:0.62130
[9]	validation_0-mlogloss:0.59124
[10]	validation_0-mlogloss:0.57919
[11]	validation_0-mlogloss:0.56239
[12]	validation_0-mlogloss:0.55463
[13]	validation_0-mlogloss:0.54635
[14]	validation_0-mlogloss:0.54077
[15]	validation_0-mlogloss:0.53354
[16]	validation_0-mlogloss:0.53134
[17]	validation_0-mlogloss:0.52404
[18]	validation_0-mlogloss:0.51806
[19]	validation_0-mlogloss:0.51972
[20]	validation_0-mlogloss:0.52043
[21]	validation_0-mlogloss:0.52236
[22]	validation_0-mlogloss:0.52277
[23]	validation_0-mlogloss:0.52186
[24]	validation_0-mlogloss:0.52557
[25]	validation_0-mlogloss:0.52795
[26]	validation_0-mlogloss:0.52794
[27]	validation_0-mlogloss:0.52859
[28]	validation_0-mlogloss:0.5

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', random_state=777,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [331]:
y_pred = model.predict(X_val)
y_check = np.array(y_val)

In [332]:
accuracy_score(y_pred, y_check)

0.8127853881278538

In [333]:
model.feature_importances_

array([0.29278737, 0.18150635, 0.3992788 , 0.05933257, 0.01255671,
       0.01184167, 0.00995908, 0.01382609, 0.01891138], dtype=float32)

In [334]:
for i in range(0, 100):
    
    if y_check[i] != y_pred[i]: 
        print(y_check[i]-y_pred[i], ': ', y_check[i]+1, y_pred[i]+1)

-1 :  6 7
-1 :  6 7
-1 :  8 9
-1 :  6 7
-1 :  1 2
-1 :  2 3
-1 :  8 9
-1 :  5 6
-1 :  11 12
-1 :  7 8
-11 :  1 12
-1 :  6 7
1 :  2 1
-1 :  4 5
11 :  12 1
-5 :  4 9
10 :  11 1


In [342]:
# 모델 저장
import pickle
import joblib

joblib.dump(model, 'month_xgb_200120.pkl')

['month_xgb_200120.pkl']

In [335]:
model = lgbm.LGBMClassifier(objective='multi:softmax', eval_metric='mlogloss', n_estimators=1000, random_state=SEED)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=200)

[LightGBM] [Warning] Unknown parameter: eval_metric
[1]	valid_0's multi_logloss: 1.86181
Training until validation scores don't improve for 200 rounds
[2]	valid_0's multi_logloss: 1.57401
[3]	valid_0's multi_logloss: 1.37339
[4]	valid_0's multi_logloss: 1.23333
[5]	valid_0's multi_logloss: 1.12162
[6]	valid_0's multi_logloss: 1.02621
[7]	valid_0's multi_logloss: 0.944687
[8]	valid_0's multi_logloss: 0.878383
[9]	valid_0's multi_logloss: 0.825481
[10]	valid_0's multi_logloss: 0.778912
[11]	valid_0's multi_logloss: 0.741576
[12]	valid_0's multi_logloss: 0.711715
[13]	valid_0's multi_logloss: 0.683545
[14]	valid_0's multi_logloss: 0.662673
[15]	valid_0's multi_logloss: 0.63709
[16]	valid_0's multi_logloss: 0.6188
[17]	valid_0's multi_logloss: 0.599844
[18]	valid_0's multi_logloss: 0.584152
[19]	valid_0's multi_logloss: 0.572958
[20]	valid_0's multi_logloss: 0.560826
[21]	valid_0's multi_logloss: 0.550372
[22]	valid_0's multi_logloss: 0.546217
[23]	valid_0's multi_logloss: 0.538375
[24]	va

LGBMClassifier(eval_metric='mlogloss', n_estimators=1000,
               objective='multi:softmax', random_state=777)

In [336]:
y_pred = model.predict(X_val)
y_check = np.array(y_test)
accuracy_score(y_pred, y_check)

0.7853881278538812

In [337]:
for i in range(0, 100):
    
    if y_check[i] != y_pred[i]: 
        print(y_check[i]-y_pred[i], ': ', y_check[i]+1, y_pred[i]+1)

-1 :  6 7
-1 :  6 7
1 :  8 7
-1 :  8 9
-1 :  6 7
-1 :  1 2
-1 :  2 3
-1 :  8 9
-1 :  11 12
-1 :  7 8
10 :  11 1
-1 :  6 7
1 :  6 5
-1 :  4 5
11 :  12 1
-5 :  4 9
-1 :  5 6
10 :  11 1
-1 :  1 2


In [220]:
model.feature_importances_

array([1244, 1203, 1936, 7218, 4928, 3256, 4271, 4195, 4831])

In [222]:
X_train.columns

Index(['sunrise', 'sunset', 'sunnyTime', 'T_mean', 'DHI_mean', 'DNI_mean',
       'WS_mean', 'RH_mean', 'TARGET_mean'],
      dtype='object')

### 원래 있었던 변수만 사용

In [223]:
X = train_by_day.drop(['Day', 'date', 'month','sunrise', 'sunset', 'sunnyTime'], axis=1)
y = train_by_day['month']

In [224]:
SEED = 777
TEST_RATIO = 0.2

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_RATIO, random_state=SEED)

In [225]:
model = xgb.XGBClassifier(use_label_encoder=False, objective='multi:softmax', eval_metric='mlogloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [226]:
y_pred = model.predict(X_val)
y_check = np.array(y_val)
accuracy_score(y_pred, y_check)

0.6210045662100456

In [227]:
for i in range(0, 100):
    
    if y_check[i] != y_pred[i]: 
        print(y_check[i]-y_pred[i], ': ', y_check[i]+1, y_pred[i]+1)

-2 :  6 8
-1 :  6 7
1 :  5 4
11 :  12 1
11 :  12 1
-1 :  6 7
-1 :  3 4
7 :  10 3
-1 :  8 9
1 :  2 1
-1 :  6 7
1 :  8 7
-3 :  6 9
-9 :  1 10
1 :  3 2
1 :  9 8
-1 :  2 3
-11 :  1 12
1 :  7 6
-9 :  2 11
-1 :  11 12
10 :  11 1
-1 :  11 12
-11 :  1 12
1 :  9 8
-1 :  6 7
1 :  8 7
1 :  4 3
1 :  2 1
-1 :  4 5
1 :  11 10
-2 :  8 10
1 :  8 7
-11 :  1 12
-4 :  5 9
-7 :  3 10
1 :  11 10
-1 :  11 12
-11 :  1 12
-1 :  7 8


In [228]:
model = lgbm.LGBMClassifier(objective='multi:softmax', eval_metric='mlogloss')
model.fit(X_train, y_train)

[LightGBM] [Warning] Unknown parameter: eval_metric


LGBMClassifier(eval_metric='mlogloss', objective='multi:softmax')

In [230]:
y_pred = model.predict(X_val)
y_check = np.array(y_test)
accuracy_score(y_pred, y_check)

0.6164383561643836

In [231]:
for i in range(0, 100):
    
    if y_check[i] != y_pred[i]: 
        print(y_check[i]-y_pred[i], ': ', y_check[i]+1, y_pred[i]+1)

-2 :  6 8
-1 :  6 7
1 :  5 4
-2 :  6 8
11 :  12 1
-1 :  6 7
-6 :  3 9
-1 :  8 9
1 :  2 1
-1 :  6 7
1 :  8 7
-3 :  6 9
-10 :  1 11
1 :  3 2
1 :  9 8
-1 :  2 3
-11 :  1 12
-9 :  2 11
10 :  11 1
10 :  11 1
-1 :  11 12
-10 :  1 11
1 :  9 8
-1 :  6 7
1 :  6 5
1 :  8 7
1 :  4 3
-10 :  2 12
-1 :  4 5
1 :  11 10
-2 :  8 10
-11 :  1 12
-5 :  5 10
-5 :  4 9
-1 :  5 6
-1 :  3 4
1 :  11 10
10 :  11 1
-11 :  1 12
1 :  5 4
-1 :  7 8
